### Import Libraries 

In [2]:
myID = 'ZJQFRXBWN0TFN4RFQZXCNP2YFZAX2H4NY40MBRGTGVLPVWCS'
mySecret = 'KP3R14XHTDRRW0I2IAEVWSA1EFKLNPDWB1ENNNNWNOUGJJUC'

In [3]:
CLIENT_ID = myID # your Foursquare ID
CLIENT_SECRET = mySecret # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZJQFRXBWN0TFN4RFQZXCNP2YFZAX2H4NY40MBRGTGVLPVWCS
CLIENT_SECRET:KP3R14XHTDRRW0I2IAEVWSA1EFKLNPDWB1ENNNNWNOUGJJUC


In [9]:
#Get geographical coordinates of Indy
latitude = 39.7684
longitude = -86.1581

#### Neighborhood Data of Indianapolis

In [634]:
#import csv file of indy neighborhoods
df = pd.read_csv('/Users/stephaniedudley/Downloads/IN_Neigh_B.csv')


#### Cleaning some data

In [635]:
df['Neighborhood'] = df['Neighborhood'].map(lambda x: x.strip(', Indiana'))

In [636]:
df.rename(columns ={'Latitutude': 'Latitude'},inplace =True)  

In [670]:

df.head()

,Neighborhood,Latitude,Longitude
0,Delaware Trails,39.880700,-86.182105
1,Devingto,39.834367,-86.075299
2,Sunshine Gardens,39.688101,-86.211099
3,Stout Fiel,39.748306,-86.235409
4,Park Fletcher,39.734973,-86.252495


In [639]:
# create map of Indy using latitude and longitude values
map_indy = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_indy)  
    
map_indy

In [640]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [641]:
#Now, we are ready to clean the json and structure it into a pandas dataframe.
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arthur M. Glick JCC,Gym / Fitness Center,39.879291,-86.172971
1,Holliday Park,Park,39.871231,-86.162312
2,Meridian Hills Country Club,Golf Course,39.882437,-86.162923
3,Northside Kitchenette,Sandwich Place,39.874313,-86.145419
4,Indianapolis Art Center,Art Gallery,39.877692,-86.143686


### Let's repeat process for all neighborhoods!

In [595]:
## Create function to repeat the same process to all the neighborhoods in indy
def getNearbyVenues(names, latitudes, longitudes, radius=1600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [596]:
indy_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Delaware Trails
Devingto
Sunshine Gardens
Stout Fiel
Park Fletcher
West Indianapolis
Near Westside
Southern Dunes
West Newto
Allisonville
Ameriplex
Devonshire
Downtow
Fountain Square
Northwest High School
Meridian Kessler
Nor
Maywoo
Arlington Woods
Augusta / New August
Beech Grove
St Vincent / Greenbriar
Key Meadows
North Perry
Mars Hill
Garfield Park
College Park
Geist
Lawrence
East Warre
Brendonwoo
Broad Ripple
Butler-Tarkington/Rocky Ripple
Forest Manor
Crown Hill
Glendale
Christian Park
Lawrence-Fort Ben-Oaklando
rvingto
Brightwoo
Meadows
Meridian Hills/Williams Creek
Camby
Castleto
Chapel Hill / Ben Davis
Millersville
East Gate
Raymond Park
Southeast Warre
North Central
Mapleto
Near NW - Riverside
Marian - Cold Springs
Guion Creek
Ravenswoo
Keystone at the Crossing
Canterbury-Chatar
ternational Marketplace
Speedway
Garden City
Five Points
Poplar Grove
Wanamaker
Clermont
Traders Point
South Frankl
University Heights
Edgewoo
Crooked Creek
Wynnedale / Spring Hill
Crows Nest
Homecroft

In [597]:
indy_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Delaware Trails,39.880700,-86.182105,Arthur M. Glick JCC,39.879291,-86.172971,Gym / Fitness Center
1,Delaware Trails,39.880700,-86.182105,Papa John's Pizza,39.872270,-86.178860,Pizza Place
2,Delaware Trails,39.880700,-86.182105,Juan Soloman Park,39.867663,-86.188041,Park
3,Delaware Trails,39.880700,-86.182105,INDY LOCK N KEY,39.878800,-86.182800,Locksmith
4,Delaware Trails,39.880700,-86.182105,Westlane Trails,39.884096,-86.187619,Baseball Field
5,Delaware Trails,39.880700,-86.182105,Bureau of Jewish Education,39.880281,-86.173144,Dance Studio
6,Delaware Trails,39.880700,-86.182105,W 73rd St & Hoover Rd,39.886599,-86.174476,Intersection
7,Delaware Trails,39.880700,-86.182105,Jewish Community Center Softball Fields,39.878500,-86.171403,Baseball Field
8,Delaware Trails,39.880700,-86.182105,Hoover Rd. & 64th St.,39.872096,-86.174541,Intersection
9,Delaware Trails,39.880700,-86.182105,"King Tree Services & Landscape, LLC",39.884272,-86.197644,Construction & Landscaping


In [645]:
indy_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ameriplex,9,9,9,9,9,9
Arlington Woods,32,32,32,32,32,32
Augusta / New August,29,29,29,29,29,29
Beech Grove,33,33,33,33,33,33
Brendonwoo,5,5,5,5,5,5
Brightwoo,13,13,13,13,13,13
Broad Ripple,41,41,41,41,41,41
Butler-Tarkington/Rocky Ripple,26,26,26,26,26,26
Camby,6,6,6,6,6,6


In [599]:
#Finds out how many unique category of venues exist
print('There are {} uniques categories.'.format(len(indy_venues['Venue Category'].unique())))

There are 286 uniques categories.


### Anaylyze Neighborhoods

In [600]:
# one hot encoding
indy_onehot = pd.get_dummies(indy_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
indy_onehot['Neighborhood'] = indy_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [indy_onehot.columns[-1]] + list(indy_onehot.columns[:-1])
indy_onehot = indy_onehot[fixed_columns]

indy_onehot.head(20)

,Neighborhood,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Gym,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Elementary School,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Print Shop,Pub,Racecourse,Racetrack,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yo

In [601]:
#lets group rows by neighborhood and by taking the mean of the frequency of occurence of each category
indy_grouped = indy_onehot.groupby('Neighborhood').mean().reset_index()
indy_grouped

,Neighborhood,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bike Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Gym,College Quad,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Elementary School,Ethiopian Restaurant,Event Service,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paintball Field,Paper / Office Supplies Store,Park,Pastry Shop,Pawn Shop,Performing Arts Venue,Pet Service,Pet Store,Pharmacy,Photography Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Print Shop,Pub,Racecourse,Racetrack,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Weight Loss Center,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yo

In [602]:
indy_grouped.shape

(76, 287)

In [603]:
#print out each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in indy_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = indy_grouped[indy_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ameriplex----
                        venue  freq
0         American Restaurant  0.33
1                       Hotel  0.33
2        Gym / Fitness Center  0.11
3  Construction & Landscaping  0.11
4              Sandwich Place  0.11


----Arlington Woods----
                    venue  freq
0    Fast Food Restaurant  0.09
1            Intersection  0.09
2      Seafood Restaurant  0.06
3  Furniture / Home Store  0.06
4             Pizza Place  0.06


----Augusta / New August----
                  venue  freq
0        Discount Store  0.10
1           Pizza Place  0.10
2    Mexican Restaurant  0.10
3  Fast Food Restaurant  0.10
4                  Bank  0.07


----Beech Grove----
         venue  freq
0  Gas Station  0.09
1         Park  0.09
2         Bank  0.09
3     Pharmacy  0.06
4          Bar  0.06


----Brendonwoo----
           venue  freq
0    Golf Course   0.2
1     Sports Bar   0.2
2  Moving Target   0.2
3          Trail   0.2
4           Lake   0.2


----Brightwoo----
          

In [604]:
#Let's put into Pandas df
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [768]:
#create new df and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = indy_grouped['Neighborhood']

for ind in np.arange(indy_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(indy_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ameriplex,American Restaurant,Hotel,Gym / Fitness Center,Sandwich Place,Construction & Landscaping,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
1,Arlington Woods,Fast Food Restaurant,Intersection,Pizza Place,Seafood Restaurant,Furniture / Home Store,Food,Karaoke Bar,Nightclub,Office,Bowling Alley
2,Augusta / New August,Mexican Restaurant,Discount Store,Pizza Place,Fast Food Restaurant,Bank,Park,Dog Run,Sandwich Place,Nightclub,Fried Chicken Joint
3,Beech Grove,Gas Station,Bank,Park,Pharmacy,Bar,Pet Store,Jewelry Store,Breakfast Spot,Fast Food Restaurant,Sandwich Place
4,Brendonwoo,Lake,Sports Bar,Moving Target,Trail,Golf Course,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market
5,Brightwoo,Park,Thrift / Vintage Store,Gas Station,Office,Fast Food Restaurant,Gym,Kitchen Supply Store,Fried Chicken Joint,Discount Store,Intersection
6,Broad Ripple,Gas Station,Mobile Phone Shop,Park,Art Gallery,Market,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Pool,Supermarket
7,Butler-Tarkington/Rocky Ripple,Coffee Shop,American Restaurant,Ice Cream Shop,Cupcake Shop,Jewelry Store,Supermarket,Golf Course,College Quad,Basketball Stadium,Tennis Court
8,Camby,Donut Shop,Gas Station,Pizza Place,Coffee Shop,Convenience Store,Liquor Store,Food & Drink Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market
9,Canterbury-Chatar,Italian Restaurant,Pub,Fast Food Restaurant,Sandwich Place,Gym / Fitness Center,Cosmetics Shop,Pet Store,American Restaurant,Liquor Store,Mexican Restaurant


### Kmeans Clustering Method

In [769]:
# set number of clusters
kclusters = 7

indy_grouped_clustering = indy_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state = 0).fit(indy_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans.labels_)

76

In [770]:
#create new df that includes cluster as well as th top 10 venues for each neighborhood

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

indy_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
indy_merged = indy_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

indy_merged.head(10) # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Delaware Trails,39.880700,-86.182105,1.0,Intersection,Baseball Field,Locksmith,Park,Dance Studio,Pizza Place,Gym / Fitness Center,Construction & Landscaping,Farmers Market,Fast Food Restaurant
1,Devingto,39.834367,-86.075299,0.0,Seafood Restaurant,Liquor Store,Fried Chicken Joint,Cosmetics Shop,Discount Store,Park,Pharmacy,Department Store,Gas Station,Shopping Mall
2,Sunshine Gardens,39.688101,-86.211099,1.0,Scenic Lookout,Food,Furniture / Home Store,Harbor / Marina,Rental Service,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
3,Stout Fiel,39.748306,-86.235409,5.0,Fast Food Restaurant,Pizza Place,Discount Store,Rental Car Location,Motorcycle Shop,Pharmacy,Mexican Restaurant,Men's Store,Breakfast Spot,Storage Facility
4,Park Fletcher,39.734973,-86.252495,2.0,Hotel,American Restaurant,Breakfast Spot,Sandwich Place,Salad Place,Juice Bar,Gas Station,Food,Liquor Store,Coffee Shop
5,West Indianapolis,39.775092,-86.132160,1.0,Brewery,Bar,Coffee Shop,Mexican Restaurant,American Restaurant,Pizza Place,Pub,Gastropub,Antique Shop,Comedy Club
6,Near Westside,39.779312,-86.213439,5.0,Fast Food Restaurant,Gas Station,Pawn Shop,History Museum,Park,Discount Store,Pizza Place,Dive Bar,Event Service,Sandwich Place
7,Southern Dunes,39.657838,-86.215968,1.0,Liquor Store,Home Service,Clothing Store,Golf Course,Gym,Chinese Restaurant,Sports Club,Pool,Cosmetics Shop,Discount Store
8,West Newto,39.653100,-86.282770,1.0,Moving Target,Sandwich Place,Market,Farm,Historic Site,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market
9,Allisonville,39.905665,86.079608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [771]:
indy_merged.dropna(inplace=True)

In [772]:
indy_merged.rename(columns = {'Cluster Labels':'clusterlabels'},inplace = True)

In [773]:
indy_merged.set_index("Neighborhood", inplace = True)

In [774]:
indy_merged=indy_merged.rename(index={'West Indianapolis': 'Near Eastside'})

In [775]:
indy_merged.head(20)

,Latitude,Longitude,clusterlabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,,,,
Delaware Trails,39.880700,-86.182105,1.0,Intersection,Baseball Field,Locksmith,Park,Dance Studio,Pizza Place,Gym / Fitness Center,Construction & Landscaping,Farmers Market,Fast Food Restaurant
Devingto,39.834367,-86.075299,0.0,Seafood Restaurant,Liquor Store,Fried Chicken Joint,Cosmetics Shop,Discount Store,Park,Pharmacy,Department Store,Gas Station,Shopping Mall
Sunshine Gardens,39.688101,-86.211099,1.0,Scenic Lookout,Food,Furniture / Home Store,Harbor / Marina,Rental Service,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
Stout Fiel,39.748306,-86.235409,5.0,Fast Food Restaurant,Pizza Place,Discount Store,Rental Car Location,Motorcycle Shop,Pharmacy,Mexican Restaurant,Men's Store,Breakfast Spot,Storage Facility
Park Fletcher,39.734973,-86.252495,2.0,Hotel,American Restaurant,Breakfast Spot,Sandwich Place,Salad Place,Juice Bar,Gas Station,Food,Liquor Store,Coffee Shop
Near Eastside,39.775092,-86.132160,1.0,Brewery,Bar,Coffee Shop,Mexican Restaurant,American Restaurant,Pizza Place,Pub,Gastropub,Antique Shop,Comedy Club
Near Westside,39.779312,-86.213439,5.0,Fast Food Restaurant,Gas Station,Pawn Shop,History Museum,Park,Discount Store,Pizza Place,Dive Bar,Event Service,Sandwich Place
Southern Dunes,39.657838,-86.215968,1.0,Liquor Store,Home Service,Clothing Store,Golf Course,Gym,Chinese Restaurant,Sports Club,Pool,Cosmetics Shop,Discount Store
West Newto,39.653100,-86.282770,1.0,Moving Target,Sandwich Place,Market,Farm,Historic Site,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market


In [783]:
indy_merged=indy_merged.reset_index()

In [777]:
#df[list("ABCD")] = df[list("ABCD")].astype(int)

indy_merged['clusterlabels'] = indy_merged['clusterlabels'].astype(int)

In [784]:
indy_merged

,Neighborhood,Latitude,Longitude,clusterlabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Delaware Trails,39.880700,-86.182105,1,Intersection,Baseball Field,Locksmith,Park,Dance Studio,Pizza Place,Gym / Fitness Center,Construction & Landscaping,Farmers Market,Fast Food Restaurant
1,Devingto,39.834367,-86.075299,0,Seafood Restaurant,Liquor Store,Fried Chicken Joint,Cosmetics Shop,Discount Store,Park,Pharmacy,Department Store,Gas Station,Shopping Mall
2,Sunshine Gardens,39.688101,-86.211099,1,Scenic Lookout,Food,Furniture / Home Store,Harbor / Marina,Rental Service,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
3,Stout Fiel,39.748306,-86.235409,5,Fast Food Restaurant,Pizza Place,Discount Store,Rental Car Location,Motorcycle Shop,Pharmacy,Mexican Restaurant,Men's Store,Breakfast Spot,Storage Facility
4,Park Fletcher,39.734973,-86.252495,2,Hotel,American Restaurant,Breakfast Spot,Sandwich Place,Salad Place,Juice Bar,Gas Station,Food,Liquor Store,Coffee Shop
5,Near Eastside,39.775092,-86.132160,1,Brewery,Bar,Coffee Shop,Mexican Restaurant,American Restaurant,Pizza Place,Pub,Gastropub,Antique Shop,Comedy Club
6,Near Westside,39.779312,-86.213439,5,Fast Food Restaurant,Gas Station,Pawn Shop,History Museum,Park,Discount Store,Pizza Place,Dive Bar,Event Service,Sandwich Place
7,Southern Dunes,39.657838,-86.215968,1,Liquor Store,Home Service,Clothing Store,Golf Course,Gym,Chinese Restaurant,Sports Club,Pool,Cosmetics Shop,Discount Store
8,West Newto,39.653100,-86.282770,1,Moving Target,Sandwich Place,Market,Farm,Historic Site,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market
9,Ameriplex,39.677300,-86.314908,2,American Restaurant,Hotel,Gym / Fitness Center,Sandwich Place,Construction & Landscaping,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [785]:
indy_merged.clusterlabels.value_counts()

1    31
5    29
0    10
2     3
6     1
4     1
3     1
Name: clusterlabels, dtype: int64

In [786]:
#VISUALIZE CLUSTERS!
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
colorrange = ['red', 'blue', 'green', 'purple', 'orange', 'gray','lightgreen']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(indy_merged['Latitude'], indy_merged['Longitude'], indy_merged['Neighborhood'], indy_merged['clusterlabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colorrange[cluster],
        fill=True,
        fill_color=colorrange[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [694]:
indy_merged.clusterlabels.value_counts()

1    31
5    29
0    10
2     3
6     1
4     1
3     1
Name: clusterlabels, dtype: int64

In [787]:
#Cluster 0
indy_merged.loc[indy_merged['clusterlabels'] == 0, indy_merged.columns[[1] + list(range(4, indy_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,39.834367,Seafood Restaurant,Liquor Store,Fried Chicken Joint,Cosmetics Shop,Discount Store,Park,Pharmacy,Department Store,Gas Station,Shopping Mall
28,39.789740,Golf Course,Discount Store,Intersection,Pharmacy,Gas Station,Furniture / Home Store,Bakery,Park,Trail,Yoga Studio
33,39.818525,Museum,Park,Bank,Discount Store,Gas Station,Science Museum,Seafood Restaurant,Burger Joint,Mexican Restaurant,Café
38,39.801037,Park,Thrift / Vintage Store,Gas Station,Office,Fast Food Restaurant,Gym,Kitchen Supply Store,Fried Chicken Joint,Discount Store,Intersection
49,39.819498,Discount Store,Museum,Bank,Park,Science Museum,Opera House,Gas Station,Food,Chinese Restaurant,Fried Chicken Joint
50,39.798689,Golf Course,Park,Outdoor Sculpture,Tennis Court,Basketball Court,Bar,Thrift / Vintage Store,Bakery,Motorsports Shop,Brewery
51,39.816170,Golf Course,Video Store,Pharmacy,American Restaurant,Grocery Store,Park,Hospital,Theater,Lake,College Gym
63,39.893653,Pool,Hot Dog Joint,Farm,Park,Construction & Landscaping,Sculpture Garden,Flea Market,Falafel Restaurant,Farmers Market,Fast Food Restaurant
68,39.819328,Museum,Park,Art Museum,Historic Site,American Restaurant,Discount Store,Café,Gas Station,Bank,Science Museum
69,39.819328,Museum,Park,Art Museum,Historic Site,American Restaurant,Discount Store,Café,Gas Station,Bank,Science Museum


In [788]:
#Cluster 1

indy_merged.loc[indy_merged['clusterlabels'] == 1, indy_merged.columns[[0] + list(range(4, indy_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Delaware Trails,Intersection,Baseball Field,Locksmith,Park,Dance Studio,Pizza Place,Gym / Fitness Center,Construction & Landscaping,Farmers Market,Fast Food Restaurant
2,Sunshine Gardens,Scenic Lookout,Food,Furniture / Home Store,Harbor / Marina,Rental Service,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
5,Near Eastside,Brewery,Bar,Coffee Shop,Mexican Restaurant,American Restaurant,Pizza Place,Pub,Gastropub,Antique Shop,Comedy Club
7,Southern Dunes,Liquor Store,Home Service,Clothing Store,Golf Course,Gym,Chinese Restaurant,Sports Club,Pool,Cosmetics Shop,Discount Store
8,West Newto,Moving Target,Sandwich Place,Market,Farm,Historic Site,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market
11,Downtow,Hotel,Bar,American Restaurant,Pizza Place,Steakhouse,Mexican Restaurant,Burger Joint,Sandwich Place,Breakfast Spot,Cocktail Bar
12,Fountain Square,Bar,Brewery,Gas Station,Mexican Restaurant,Discount Store,Pizza Place,Art Gallery,Sandwich Place,Clothing Store,Music Venue
14,Meridian Kessler,Pizza Place,Ice Cream Shop,Pub,Italian Restaurant,Coffee Shop,Breakfast Spot,Gym,Jewelry Store,Deli / Bodega,Liquor Store
15,Nor,Hotel,Intersection,Gym,Deli / Bodega,Pool,Pub,Rental Car Location,Convenience Store,Park,Bakery
17,Arlington Woods,Fast Food Restaurant,Intersection,Pizza Place,Seafood Restaurant,Furniture / Home Store,Food,Karaoke Bar,Nightclub,Office,Bowling Alley


In [697]:
#Cluster 2
indy_merged.loc[indy_merged['clusterlabels'] == 2, indy_merged.columns[[0] + list(range(4, indy_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Park Fletcher,Hotel,American Restaurant,Breakfast Spot,Sandwich Place,Salad Place,Juice Bar,Gas Station,Food,Liquor Store,Coffee Shop
10,Ameriplex,American Restaurant,Hotel,Gym / Fitness Center,Sandwich Place,Construction & Landscaping,Yoga Studio,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
53,Guion Creek,Hotel,Food Truck,Gym,Outdoors & Recreation,Music Venue,Clothing Store,Breakfast Spot,Office,Basketball Court,Café


In [698]:
#Cluster 3
indy_merged.loc[indy_merged['clusterlabels'] == 3, indy_merged.columns[[0] + list(range(4, indy_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Clermont,Bar,Furniture / Home Store,Bowling Alley,Baseball Field,Frozen Yogurt Shop,Fried Chicken Joint,Gaming Cafe,French Restaurant,Fountain,Garden Center


In [699]:
#Cluster 4
indy_merged.loc[indy_merged['clusterlabels'] == 4, indy_merged.columns[[0] + list(range(4, indy_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
60,Five Points,Bar,American Restaurant,Pizza Place,General Entertainment,Food & Drink Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop


In [700]:
#Cluster 5
indy_merged.loc[indy_merged['clusterlabels'] == 5, indy_merged.columns[[0] + list(range(4, indy_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Stout Fiel,Fast Food Restaurant,Pizza Place,Discount Store,Rental Car Location,Motorcycle Shop,Pharmacy,Mexican Restaurant,Men's Store,Breakfast Spot,Storage Facility
6,Near Westside,Fast Food Restaurant,Gas Station,Pawn Shop,History Museum,Park,Discount Store,Pizza Place,Dive Bar,Event Service,Sandwich Place
11,Devonshire,Fast Food Restaurant,Gas Station,Fried Chicken Joint,Pharmacy,Discount Store,Intersection,Other Repair Shop,Building,Mexican Restaurant,Breakfast Spot
14,Northwest High School,Pizza Place,Park,Discount Store,American Restaurant,Fast Food Restaurant,Mexican Restaurant,Grocery Store,Gym / Fitness Center,Video Store,Sandwich Place
17,Maywoo,Pizza Place,Gym,Gas Station,Fast Food Restaurant,Bank,Chinese Restaurant,Optical Shop,Discount Store,Diner,French Restaurant
19,Augusta / New August,Mexican Restaurant,Discount Store,Pizza Place,Fast Food Restaurant,Bank,Park,Dog Run,Sandwich Place,Nightclub,Fried Chicken Joint
20,Beech Grove,Gas Station,Bank,Park,Pharmacy,Bar,Pet Store,Jewelry Store,Breakfast Spot,Fast Food Restaurant,Sandwich Place
23,North Perry,Mexican Restaurant,Pizza Place,Gas Station,Bank,Discount Store,Fast Food Restaurant,Clothing Store,Sandwich Place,American Restaurant,Bakery
24,Mars Hill,Pharmacy,Pizza Place,Liquor Store,Fast Food Restaurant,Discount Store,Mexican Restaurant,Gas Station,Intersection,Gym / Fitness Center,Golf Course
25,Garfield Park,Grocery Store,Pizza Place,Fast Food Restaurant,Park,American Restaurant,Discount Store,Concert Hall,Brewery,Garden,Fried Chicken Joint


In [701]:
#Cluster 6
indy_merged.loc[indy_merged['clusterlabels'] == 6, indy_merged.columns[[0] + list(range(4, indy_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Brendonwoo,Lake,Sports Bar,Moving Target,Trail,Golf Course,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market


### Analyze Breweries in Neighborhoods Close to Lucas Oil Stadium

From analyzing the map above, the three nearest neighborhoods are Downtown, Fountain Square, and West Indianapolis

In [702]:
nearby_grouped_all = indy_venues[(indy_venues['Neighborhood'].isin(['Downtow','Fountain Square','West Indianapolis']))]

In [703]:
nearby_grouped_all.set_index("Neighborhood", inplace = True)

In [704]:
nearby_grouped_all=nearby_grouped_all.rename(index={'West Indianapolis': 'Near Eastside'})

In [705]:
breweries_df = nearby_grouped_all[(nearby_grouped_all['Venue Category'].isin(['Brewery']))]

In [706]:
breweries_df

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Near Eastside,39.775092,-86.132160,Flat12 Bierwerks,39.772757,-86.138504,Brewery
Near Eastside,39.775092,-86.132160,Ash & Elm Cider Co,39.767869,-86.125255,Brewery
Near Eastside,39.775092,-86.132160,Centerpoint Brewing,39.782138,-86.137135,Brewery
Near Eastside,39.775092,-86.132160,Sun King Brewery,39.769116,-86.144946,Brewery
Near Eastside,39.775092,-86.132160,Indiana City Brewing Co,39.766130,-86.141270,Brewery
Near Eastside,39.775092,-86.132160,St Joseph Brewery & Public House,39.774969,-86.145639,Brewery
Near Eastside,39.775092,-86.132160,outliers brewing company,39.775179,-86.147450,Brewery
Downtow,39.768400,-86.157875,Rock Bottom Restaurant & Brewery,39.767377,-86.158554,Brewery
Fountain Square,39.756585,-86.133249,New Day Craft Mead & Cider,39.752406,-86.139755,Brewery
